# Домашнее задание № 1. Определение тональности отзывов
### Елизавета Клыкова, БКЛ181
## Пункт 1: сбор данных
Для сбора материалов был выбран сайт https://bookmix.ru/ с отзывами на книги. С помощью краулера получим положительные и отрицательные отзывы на различные произведения.

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import collections
import nltk
import re
import requests
import string
from bs4 import BeautifulSoup
from collections import OrderedDict
from nltk import *
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import accuracy_score
from string import punctuation
from tqdm.auto import tqdm
morph = MorphAnalyzer()
session = requests.session()

Сначала получим с основной страницы список ссылок на различные страницы с отзывами, затем соберем с этих страниц отзывы с рейтингом 1-2 или 5 звезд. (Для отрицательных возьмем оценки 1 или 2 звезды, т.к. отзывов с 1 звездой слишком мало.)

*Простите за длинную строку, у меня все падает, когда пытаюсь ее переносить каким-либо способом :(*

In [3]:
def get_links():
    begin_with = 0  # индекс страницы в ссылке
    links = []
    for i in tqdm(range(20)):
        url = f'https://bookmix.ru/comments/index.phtml?begin={begin_with}&num_point=20&num_points=20.html'
        page = session.get(url).text
        soup = BeautifulSoup(page, 'html.parser')
        for block in soup.find_all('h5'):
            link = block.find('a').attrs['href']
            if link.startswith('/discussion'):
                full_link = 'https://bookmix.ru' + link
                links.append(full_link)
        begin_with += 20
    return links

5:80: E501 line too long (107 > 79 characters)


Теперь, проходя по полученным ссылкам, будем собирать полные тексты отзывов, если рейтинг нам подходит (1-2 или 5 звезд).

In [4]:
def get_reviews(links):
    good_reviews = []
    bad_reviews = []
    for link in tqdm(links):
        url = f'{link}.html'
        page = session.get(url)
        page.encoding = 'utf-8'
        soup = BeautifulSoup(page.text, 'html.parser')
        reviews = soup.find_all('div', {'class': 'item-comment level1'})
        for review in reviews:
            rating = review.find('div', {'class': 'rating'})
            if rating:  # иногда рейтинга нет
                rating = str(rating.attrs['class'][-1][-1])
            else:
                pass
            if rating == '1' or rating == '2' or rating == '5':
                text = review.find(
                    'div', {'class': 'comment-content'}).find('p').text
                if rating == '5':
                    good_reviews.append(text)
                else:
                    bad_reviews.append(text)
    return good_reviews, bad_reviews

In [5]:
good_reviews, bad_reviews = get_reviews(get_links())

Проверим, что получилось:

In [6]:
len(good_reviews)

3747

In [7]:
len(bad_reviews)

301

Сохраним полученные отзывы в два отдельных файла. В качестве разделителя будем использовать три звездочки, чтобы при необходимости можно было прочитать файл и корректно разделить отзывы.

In [8]:
def save_reviews(filename, reviews):
    with open(filename, 'w', encoding='utf-8') as f:
        full_text = '***'.join(reviews)
        f.write(full_text)

In [9]:
save_reviews('good_reviews.txt', good_reviews)
save_reviews('bad_reviews.txt', bad_reviews)

## Пункт 2: обработка данных
Токенизируем слова, приведем к нижнему регистру и к начальной форме. Поскольку NLTK считает пунктуацию за слова, заменим знаки препинания (кроме дефиса) на пробелы и только потом применим word_tokenize.

Посколько отрицательных отзывов всегда получается меньше, чем положительных, поступим так: возьмем для обучения количество плохих отзывов минус 5 (т.к. по 5 отзывов нам нужно для проверки). Можно взять фиксированное число, но первый способ надежнее, т.к. при новом запуске программы число плохих отзывов может как увеличиваться, так и уменьшаться.

Я буду считывать отзывы из созданных ранее файлов (это удобнее, если не хочется запускать всю программу целиком).

In [10]:
with open('good_reviews.txt', encoding='utf-8') as f:
    good_reviews = f.read().split('***')
with open('bad_reviews.txt', encoding='utf-8') as f1:
    bad_reviews = f1.read().split('***')

In [11]:
rnum = len(bad_reviews) - 5
good_text = '\n'.join(good_reviews[0:rnum])
bad_text = '\n'.join(bad_reviews[0:rnum])

In [12]:
def tokenize_text(text):
    punct = list(re.sub('-', '', string.punctuation))
    other_punct = ['``', "\'\'", '...', '--', 'https', '–',
                   '—', '«', '»', '“', '”', '’', '***', '…']
    rx = '[' + re.escape(''.join(punct + other_punct)) + ']'
    word_list = word_tokenize((re.sub(rx, ' ', text)).lower())
    lemmas = [morph.parse(w)[0].normal_form for w in word_list]
    return lemmas

In [13]:
good_words = tokenize_text(good_text)
bad_words = tokenize_text(bad_text)

## Пункт 3: составление тонального словаря

Для каждого типа отзывов создадим частотный словарь и получим множество слов, частотность которых больше 3.

In [14]:
def make_freqlist(words):
    freqlist = Counter()
    for word in words:
        if word.isalpha():
            freqlist[word] += 1
    freq_words = [k for k in list(freqlist.keys())
                  if freqlist[k] > 2]
    return set(freq_words)

In [15]:
good_freqlist = make_freqlist(good_words)
bad_freqlist = make_freqlist(bad_words)

Посмотрим, что получилось:

In [16]:
len(good_freqlist)

1224

In [17]:
len(bad_freqlist)

1217

Теперь найдем пересечение полученных множеств и вычтем его из каждого из них.

In [18]:
common_words = good_freqlist.intersection(bad_freqlist)
good_freqlist.difference_update(common_words)
bad_freqlist.difference_update(common_words)

Посмотрим, сколько слов осталось в каждом множестве:

In [19]:
len(good_freqlist)

527

In [20]:
len(bad_freqlist)

520

## Пункт 4: определение тональности отзыва

Токенизируем тестовые отзывы, а затем, перебирая слова в каждом отзыве, посчитаем, сколько слов положительной или отрицательной тональности встретилось в отзыве, будем определять, к какому типу относится этот отзыв. 

In [21]:
good_test_texts = [tokenize_text(t) for t in good_reviews[-5:]]
bad_test_texts = [tokenize_text(t) for t in bad_reviews[-5:]]

In [22]:
def detect_review_type(good_freqlist, bad_freqlist, review):
    review_freqlist = Counter(review)
    good_words = 0
    bad_words = 0
    for k in list(review_freqlist.keys()):
        if k in good_freqlist:
            good_words += int(review_freqlist[k])
        elif k in bad_freqlist:
            bad_words += int(review_freqlist[k])
    if good_words > bad_words:
        result = 'good'
    elif bad_words > good_words:
        result = 'bad'
    else:
        result = 'unknown'
    return result

Теперь проверим работу программы на двух типах отзывов отдельно.

In [23]:
results = {}
for i, text in enumerate(good_test_texts):
    results[str(i+1)+' good review'] = detect_review_type(
        good_freqlist, bad_freqlist, text)
for i, text in enumerate(bad_test_texts):
    results[str(i+1)+' bad review'] = detect_review_type(
        good_freqlist, bad_freqlist, text)

In [24]:
results

{'1 good review': 'good',
 '2 good review': 'good',
 '3 good review': 'good',
 '4 good review': 'good',
 '5 good review': 'good',
 '1 bad review': 'bad',
 '2 bad review': 'bad',
 '3 bad review': 'bad',
 '4 bad review': 'bad',
 '5 bad review': 'good'}

Наконец, подсчитаем accuracy:

In [25]:
def test_detect_type(good_test_texts, bad_test_texts):
    results = []  # тип отзывы по мнению программы
    real_type = []  # верный тип отзыва
    for text in good_test_texts:
        predicted_type = detect_review_type(
            good_freqlist, bad_freqlist, text)
        results.append(predicted_type)
        real_type.append('good')
    for text in bad_test_texts:
        predicted_type = detect_review_type(
            good_freqlist, bad_freqlist, text)
        results.append(predicted_type)
        real_type.append('bad')
    return accuracy_score(results, real_type)

In [26]:
test_detect_type(good_test_texts, bad_test_texts)

0.9

## Пункт 5: улучшение программы

1 способ. В данном случае при определении тональности мы считали, что все слова, встретившиеся в том или ином тональном словаре, имеют одинаковый вес. Для улучшения точности можно было бы установить разный вес для слов, которые встречаются в том или ином типе отзывов очень часто, и более редких слов. Таким образом, слова, которые встречаются в отзывах определенного типа очень часто, влияли бы на результат больше, чем слова, которые в этом типе отзывов встречаются редко.

2 способ. Мы использовали для определения тональности слова, но это не очень точный способ, поскольку такие показательные лексемы, как "нравиться", "впечатлять", могли встретиться в обоих типах текстов, а значит, не попали в тональные словари. Этого можно было бы избежать, используя биграммы: тогда биграмма "не нравиться" точно встречалась бы в отрицательных отзывых и, скорее всего, не встречалась бы в положительных. Но для того чтобы реализовать этот способ, нужна бОльшая выборка.

3 способ. Поскольку наша выборка довольно маленькая, неплохо было бы еще на этапе сбора данных добавить какую-нибудь проверку, что полученный отзыв действительно соответствует заявленному рейтингу. Бывает так, что человек ошибается и пишет плохой отзыв, а потом случайно ставит 5 звезд, и наоборот (это, кстати, может влиять и на accuracy, если при проверке попадется такой отзыв с несоответствующим рейтингом). Конечно, отчасти от такого "шума" спасает отсечение редких слов при создании множеств, но нет никакой гарантии, что в каком-нибудь отзыве некоторое слово не встретится три раза и не испортит общую картину, попав не в тот словарь. На маленьких объемах данных такую проверку можно выполнить и вручную, а вот на более крупных становится сложнее: нужно как-то сравнивать каждый отзыв с другими отзывами той же тональности. Зато чем больше данных, тем меньше влияют ошибочно попавшие не в то множество отзывы: можно просто установить более жесткий фильтр по частотности, и они отсеются.